# Monkey tests (naive tests)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import sys
import os
sys.path.append(os.path.abspath('../common/'))
import common_functions as cf

In [2]:
weather_path = 'Input/weather_train_set0.csv'
demand_path = 'Input/demand_train_set0.csv'
solar_path = 'Input/pv_train_set0.csv'
dp = cf.DataPreprocesser(weather_path, demand_path, solar_path)
dp.load_df()
dp.remove_nan()
dp.interpolate_df()
dp.set_df(dp.df[dp.df['week']!=44])
dp.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1,week,dow,hour,sp
datetime,,,,,,,,,,,,,,,,,,,,
2017-11-06 00:00:00,2.32,0.0,0.0,1.61,5.14,10.76,6.350,3.58,11.120,7.390,0.0,0.0,0.0,0.0,0.0,0.0,45,0,0,1.0
2017-11-06 00:30:00,2.22,0.0,0.0,2.98,5.05,10.70,6.245,3.58,11.060,7.325,0.0,0.0,0.0,0.0,0.0,0.0,45,0,0,2.0
2017-11-06 01:00:00,2.04,0.0,0.0,2.76,4.96,10.64,6.140,3.58,11.000,7.260,0.0,0.0,0.0,0.0,0.0,0.0,45,0,1,3.0
2017-11-06 01:30:00,1.97,0.0,0.0,3.98,5.03,10.60,6.085,3.57,10.965,7.255,0.0,0.0,0.0,0.0,0.0,0.0,45,0,1,4.0
2017-11-06 02:00:00,1.91,0.0,0.0,2.13,5.10,10.56,6.030,3.56,10.930,7.250,0.0,0.0,0.0,0.0,0.0,0.0,45,0,2,5.0


In [3]:
prediction_week = 30

## Test 1 : take demand of previous week, solar of previous week, charge max for battery

In [4]:
mp = cf.MLPredictor(dp,prediction_week)
predicted_demand = mp.get_demand_previous_week()

/home/raph/project/westernpower/westernpower/task0/common_functions.py:254: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  demand_prediction['week']=demand_prediction.index.week


In [5]:
predicted_solar_power = dp.df.loc[dp.df['week'] == (prediction_week-1), ['pv_power_mw']]
predicted_solar_power.index = predicted_solar_power.index + pd.DateOffset(7)
predicted_solar_power

,pv_power_mw
datetime,
2018-07-23 00:00:00,0.0
2018-07-23 00:30:00,0.0
2018-07-23 01:00:00,0.0
2018-07-23 01:30:00,0.0
2018-07-23 02:00:00,0.0
...,...
2018-07-29 21:30:00,0.0
2018-07-29 22:00:00,0.0
2018-07-29 22:30:00,0.0


In [6]:
predicted_data = predicted_demand.merge(predicted_solar_power,how='outer', left_index=True, right_index=True)
predicted_data

,demand_MW,week,dow,hour,sp,pv_power_mw
datetime,,,,,,
2018-07-23 00:00:00,1.73,30,0,0,1.0,0.0
2018-07-23 00:30:00,1.62,30,0,0,2.0,0.0
2018-07-23 01:00:00,1.56,30,0,1,3.0,0.0
2018-07-23 01:30:00,1.48,30,0,1,4.0,0.0
2018-07-23 02:00:00,1.48,30,0,2,5.0,0.0
...,...,...,...,...,...,...
2018-07-29 21:30:00,2.40,30,6,21,44.0,0.0
2018-07-29 22:00:00,2.15,30,6,22,45.0,0.0
2018-07-29 22:30:00,1.93,30,6,22,46.0,0.0


In [7]:
bpd = cf.BatteryPowerDispatcher
B= bpd.get_all_dispatch_in_a_week(bpd,predicted_data, prediction_week)
B

,3000,3001,3002,3003,3004,3005,3006
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.003687,0.012497,0.004397,0.00605,0.009041,0.0,0.006098


In [8]:
B.sum()

3000    0.06
3001    0.02
3002    0.02
3003    0.02
3004     0.0
3005    0.07
3006    0.08
dtype: object

In [9]:
B_final = bpd.format_dispatching_for_competition(B, predicted_data.index)
B_final.to_csv('Output/test_raph/naive1.csv')

## Test 2 : take demand of previous week, solar of previous week, charge max = solar charge max

In [10]:
mp = cf.MLPredictor(dp,prediction_week)
predicted_demand = mp.get_demand_previous_week()
predicted_solar_power = dp.df.loc[dp.df['week'] == (prediction_week-1), ['pv_power_mw']]
predicted_solar_power.index = predicted_solar_power.index + pd.DateOffset(7)
predicted_data = predicted_demand.merge(predicted_solar_power,how='outer', left_index=True, right_index=True)
predicted_data

/home/raph/project/westernpower/westernpower/task0/common_functions.py:246: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  demand_prediction['week']=demand_prediction.index.week


,demand_MW,week,dow,hour,sp,pv_power_mw
datetime,,,,,,
2018-07-23 00:00:00,1.73,30,0,0,1.0,0.0
2018-07-23 00:30:00,1.62,30,0,0,2.0,0.0
2018-07-23 01:00:00,1.56,30,0,1,3.0,0.0
2018-07-23 01:30:00,1.48,30,0,1,4.0,0.0
2018-07-23 02:00:00,1.48,30,0,2,5.0,0.0
...,...,...,...,...,...,...
2018-07-29 21:30:00,2.40,30,6,21,44.0,0.0
2018-07-29 22:00:00,2.15,30,6,22,45.0,0.0
2018-07-29 22:30:00,1.93,30,6,22,46.0,0.0


In [11]:
bpd = cf.BatteryPowerDispatcher
bpd.get_all_dispatch_in_a_week(bpd,predicted_data, prediction_week, full_solar=True)

,3000,3001,3002,3003,3004,3005,3006
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.01,0.04,0.01,0.02,0.03,0.0,0.02


## Test 3 : take demand of previous week, solar of previous day, charge max for battery

## Test 4 : take demand of previous week, solar of previous day, charge max = solar charge max